[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/audiogen-colab/blob/main/audiogen_colab.ipynb)

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install -q git+https://github.com/camenduru/audiocraft

!apt -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/audo/audiogen-medium/resolve/main/compression_state_dict.bin -d /content/model -o compression_state_dict.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/audo/audiogen-medium/resolve/main/state_dict.bin -d /content/model -o state_dict.bin

import gradio as gr
from audiocraft.models import AudioGen
from audiocraft.data.audio import audio_write

model = AudioGen.get_pretrained('/content/model')

def generate(description):
    descriptions = [description]
    output = model.generate(descriptions, progress=True)
    audio_write(f'out', output[0].cpu(), model.sample_rate, strategy="loudness", loudness_compressor=True)
    return "/content/out.wav"

block = gr.Blocks()
with block:
    with gr.Group():
      description = gr.Textbox(value="A cat meowing")
      btn = gr.Button("Generate")
      audio_out = gr.Audio()
    btn.click(generate, inputs=description, outputs=audio_out)
block.launch(debug=True)